# Import mobiles from CSV into Django `Mobile` model via API
This notebook reads `mobile_clean.csv` and sends POST requests to the running Django API endpoint to create `Mobile` records. Make sure your Django server is running (`python manage.py runserver`) before running the import cells.

In [1]:
# Required libraries
import pandas as pd
import requests
import json
import time
from typing import Dict, Any, Optional, Tuple

# API endpoint - change if your backend uses a different URL or port
API_URL = 'http://localhost:8000/api/'

# Create a single requests session to reuse connections
session = requests.Session()

In [ ]:
# Read CSV (assumes mobile_clean.csv is at the repo root)
csv_path = 'mobile_clean.csv'
try:
    df = pd.read_csv(csv_path)
    print(f'Read {len(df)} rows from {csv_path}')
    display(df.head())
except FileNotFoundError:
    print('mobile_clean.csv not found in the current working directory. Please place it in the project root or update the path.')
    df = pd.DataFrame()

Read 930 rows from mobiles_clean.csv


,Brand,Model,Mobile Weight,RAM,Front Camera,Back Camera,Processor,Battery Capacity,Screen Size,Price,Launched Year
0,Apple,iPhone 16 128GB,174.0,6,12MP,48MP,A17 Bionic,"3,600",6.1,"79,999",2024
1,Apple,iPhone 16 256GB,174.0,6,12MP,48MP,A17 Bionic,"3,600",6.1,"84,999",2024
2,Apple,iPhone 16 512GB,174.0,6,12MP,48MP,A17 Bionic,"3,600",6.1,"89,999",2024
3,Apple,iPhone 16 Plus 128GB,203.0,6,12MP,48MP,A17 Bionic,"4,200",6.7,"89,999",2024
4,Apple,iPhone 16 Plus 256GB,203.0,6,12MP,48MP,A17 Bionic,"4,200",6.7,"94,999",2024


In [4]:
def to_int(val: Any, default: Optional[int] = None) -> Optional[int]:
    try:
        if pd.isna(val):
            return default
        return int(val)
    except Exception:
        return default

def to_bool(val: Any) -> bool:
    if isinstance(val, bool):
        return val
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return False
    s = str(val).strip().lower()
    if s in ('true', 'yes', 'y', '1', 't'):
        return True
    return False

def row_to_payload(row: pd.Series) -> Dict[str, Any]:
    """Map a DataFrame row to the Mobile model payload."""
    payload: Dict[str, Any] = {}
    # String fields - only include if present and not NaN
    for col in ('brand', 'model', 'main_camera', 'sim_card', 'os', 'screen_size', 'color', 'display'):
        if col in row.index and not pd.isna(row[col]):
            payload[col] = str(row[col])
    # Boolean field
    if 'sd_card' in row.index:
        payload['sd_card'] = to_bool(row['sd_card'])
    # Integer fields
    if 'battery' in row.index:
        payload['battery'] = to_int(row['battery'])
    if 'storage' in row.index:
        payload['storage'] = to_int(row['storage'])
    if 'ram' in row.index:
        payload['ram'] = to_int(row['ram'])
    if 'self_cam' in row.index:
        payload['self_cam'] = to_int(row['self_cam'])
    if 'price' in row.index:
        payload['price'] = to_int(row['price'])
    return payload

In [5]:
def post_mobile(payload: Dict[str, Any], session: Optional[requests.Session] = None, timeout: int = 10) -> Tuple[int, Any]:
    """Post a single mobile payload to the API. Returns (status_code, response_json_or_text)."""
    s = session or requests.Session()
    try:
        resp = s.post(API_URL, json=payload, timeout=timeout)
        try:
            return resp.status_code, resp.json()
        except Exception:
            return resp.status_code, resp.text
    except Exception as e:
        return 0, str(e)

In [6]:
def import_mobiles(df: pd.DataFrame, n: Optional[int] = None, delay: float = 0.08, dry_run: bool = False):
    """Import rows from DataFrame to Django API.

























    return {'total': total, 'success': success, 'failures': failures}            print(f)        for f in failures[:10]:        print('First 10 failures:')    if failures:    print(f'Total processed: {total}, Success: {success}, Failures: {len(failures)}')    print('Import complete')        time.sleep(delay)            failures.append((idx, status, resp))        else:                print(f'Imported {success} items so far...')            if success % 50 == 0:            success += 1        if status in (200, 201):        status, resp = post_mobile(payload, session=session)            continue            print(f'DRY RUN payload for row {idx}:', payload)        if dry_run:        payload = row_to_payload(row)        total += 1    for idx, row in to_process.iterrows():    to_process = df if n is None else df.iloc[:n]    failures = []    success = 0    total = 0    df: DataFrame with rows to import
    n: optional limit of rows to process
    delay: seconds between requests
    dry_run: if True, don't actually POST, only show first payloads"""

In [ ]:
# Example: preview first 5 payloads (dry run)
if 'df' in globals() and not df.empty:
    import_mobiles(df, n=5, dry_run=True)
else:
    print('DataFrame `df` not available or empty. Run the CSV read cell first.')

In [9]:
import_mobiles(df, n=10, dry_run=True)

## Run instructions
1. Install dependencies if you don't have them:
```bash
pip install pandas requests
```
2. Make sure your Django server is running (from project root):
```bash
python manage.py runserver
```
3. In this notebook, run the cells in order. Use `import_mobiles(df, n=10, dry_run=True)` first to preview payloads. When ready, call `import_mobiles(df, n=None, delay=0.08, dry_run=False)` to POST all rows.

In [11]:
result = import_mobiles(df, n=20, dry_run=False, delay=0.08)
print(result)

None


In [ ]:
# Optional: run a small live import (uncomment to run)
# result = import_mobiles(df, n=20, dry_run=False)
# print(result)